<a href="https://colab.research.google.com/github/OANInternational/aguapp/blob/master/estructurar_bbdd_y_subir_a_firebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto para estructurar base de datos y subir a FIRBASE

# INTSTALS

In [0]:
#!pip install firebase-admin

# DIRECTORIO

utilizar google como directorio de trabajo


In [1]:
%cd

/root


In [2]:
%cd
%cd ..
%cd content

/root
/
/content


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data


In [2]:
%cd drive/'Shared drives'/'Comité de Infraestructuras, Energía y Agua'/'Proyectos'/'Red de Reparadores de Bombas Manuales'/'AGUAPP'/'BASES DE DATOS'

/content/drive/Shared drives/Comité de Infraestructuras, Energía y Agua/Proyectos/Red de Reparadores de Bombas Manuales/AGUAPP/BASES DE DATOS


In [0]:
import yaml
spreadsheet_ids = yaml.load( open("sheets_parameters.yaml", "r" ))
#bombas de 2018, ultima version
ID1=str(spreadsheet_ids['ID1'])
#bombas de 2016, ultima version
ID2=str(spreadsheet_ids['ID2'])
#bombas de 2009, ultima version
ID3=str(spreadsheet_ids['ID3'])
#bombas final, datos de 2016 y 2009
ID4=str(spreadsheet_ids['ID4'])
#bombas lisette
ID5=str(spreadsheet_ids['ID5'])
#demograpique data
ID6=str(spreadsheet_ids['ID6'])

# CONNECT TO FIREBASE

In [0]:
filename='firebase_parameters.json'

In [0]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [0]:
def on_snapshot(col_snapshot, changes, read_time):
    print(u'Callback received query snapshot.')
    print(u'Current cities in California:')
    for doc in col_snapshot:
        print(u'{}'.format(doc.id))

In [0]:
col_query = db.collection('Pompes_2019').where(u'Arrondissement', u'==', u'BIRO')

In [10]:
pumps = col_query.get()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 'Query.get' is deprecated:  please use 'Query.stream' instead.
  """Entry point for launching an IPython kernel.


In [11]:
for pump in pumps:
    print(u'{} => {}'.format(pump.id, pump.to_dict()))

Banon_1 => {'Localite': 'Banon', 'Latitude': 9.92461111111111, 'Village': 'OURAROU', 'Longitude': 2.7904722222222222, 'Marque': 'Afridev', 'Arrondissement': 'BIRO'}
Biro_1 => {'Longitude': 2.947777777777778, 'Marque': 'Vergnet', 'Arrondissement': 'BIRO', 'Localite': 'Biro', 'Village': 'BIRO', 'Latitude': 9.9}
Biro_2 => {'Arrondissement': 'BIRO', 'Localite': 'Biro', 'Latitude': 9.902777777777779, 'Village': 'BIRO', 'Longitude': 2.940638888888889, 'Marque': 'NA'}
Biro_3 => {'Arrondissement': 'BIRO', 'Localite': 'Biro', 'Village': 'BIRO', 'Latitude': 9.902944444444445, 'Longitude': 2.9468333333333336, 'Marque': 'NA'}
Biro_4 => {'Arrondissement': 'BIRO', 'Localite': 'Biro', 'Village': 'BIRO', 'Latitude': 9.903027777777778, 'Longitude': 2.9466388888888893, 'Marque': 'Afridev'}
Biro_5 => {'Longitude': 2.9463611111111114, 'Marque': 'Vergnet', 'Arrondissement': 'BIRO', 'Localite': 'Biro', 'Village': 'BIRO', 'Latitude': 9.900333333333334}
Gah_Sona_1 => {'Arrondissement': 'BIRO', 'Localite': 'Ga

# ABRIR LAS BBDD

## OPEN GOOGLE SHEETS

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
sheet_service = build('sheets', 'v4')

## IMPORTS

In [0]:
import plotly
import plotly.plotly as py
import numpy as np
import re

import pandas as pd


## OPEN FROM GOOGLE SHEET

In [0]:
ids = [ID1,ID2,ID3,ID5]
dfs = []
for ID in ids:
  #SPREADSHEET_ID = ID
  RANGE_NAME = 'Sheet1'

  gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID, range=RANGE_NAME).execute()

  dfs.append(pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0]))

df_2019=dfs[0]
df_2016=dfs[1]
df_2009=dfs[2]
df_lisette = dfs[3]

In [0]:
dfs[1]["Etat de l'ouvrage"].unique()

array(['Fonctionnel', 'Panne', 'En panne', 'Tête de pompe', '2 pompes',
       'Privé', '1  Pompe', '', 'Pompe', 'Pompe non', 'Faible débit',
       '1 pompe', 'Kalale'], dtype=object)

In [0]:
[df.columns for df in dfs]
#df_2016.head(2)
#df_2009.head(2)

[Index(['Arrondissement', 'Village', 'Localite', 'Nom_Local', 'Longitude',
        'Latitude', 'xCoord', 'yCoord', 'rqtListe_Pompe.Modele_Pompe'],
       dtype='object'),
 Index(['N°', 'Arrondissement', 'Village', 'Localité', 'Hameau',
        'Etat de l'ouvrage', 'Redevance', 'Année de réalisation', 'Marque'],
       dtype='object'),
 Index(['Arrondissement', 'Village', 'Localité', 'Population', 'Type d'ouvrage',
        'Etat', 'Longitude', 'Latitude', 'Domaine d'installation'],
       dtype='object')]

So common data is:

* 2019 vs 2016: Arrondissement, Village, Localite, Marque

* 2019 vs 2009: Arrondissement, Village, Localite, Longitude, Latitude

* 2009 vs 2016: Arrondissement, Village, Localite, Etat

ONLY DATA:

* 2019: Nom_Local which is little identifier for more info on pumps

* 2016: Année de réalisation, , Redevace, Hameau, which would be kind of the same as Nom_Local

* 2009: Population, Type d'ouvrage

## Process the Data

### FUZZY TO COMPARE Localites

In [0]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 22.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [0]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [0]:
#Example of use
choices = df_2016['Localité'].tolist()
localite = df_2019.loc[20,'Localite']
process.extract(localite, choices, limit=5)


[('Ourarou', 100),
 ('Ourarou', 100),
 ('Ourarou', 100),
 ('Ourarou Gberou kpanin', 90),
 ('Gnelkourarou', 90)]

In [0]:
#another one
word1=df_2019.loc[108,'Localite']
word2=df_2016.loc[150,'Localité']
print(word1,'\n',word2)

Kparissérou Gando 
 Kparisserou gando


In [0]:
fuzz.ratio(word1, word2)

88

In [0]:
word1=word1+'!'
fuzz.partial_ratio(word1, word2)

88

In [0]:
fuzz.ratio(word1, word2)

86

In [0]:
process.extract('Gando Gourou', df_2019['Localite'], limit=5)


[('Gando Gourou', 100, 329),
 ('Gourou', 90, 109),
 ('Gourou', 90, 110),
 ('Gourou', 90, 111),
 ('Dèh Gando', 86, 172)]

In [0]:
#last one
word1=df_2019.loc[329,'Localite']
word2=df_2016.loc[270,'Localité']
print(word1,'\n',word2)

Gando Gourou 
 Ganrou gando


In [0]:
fuzz.ratio(word1, word2)

42

In [0]:
fuzz.token_sort_ratio(word1, word2)

83

### LETS CLEAN THE DATA

#### Functions

In [0]:
#Marques des Pompes
def marques(x,name,name2):
  marque='NA'
  
  if x[name]:
    if bool(re.search('ergnet',x[name].lower())):
      marque='Vergnet'
    elif bool(re.search('ndia',x[name].lower())):
      marque='India'
    elif bool(re.search('afri',x[name].lower())):
      marque='Afridev'
  elif x[name2]:
    if bool(re.search('moteur',x[name2].lower())):
      marque='pompe_a_moteur'
  return marque

In [0]:
#Years
def getyear(x):
  year=None
  if x['Année de réalisation']:
    for s in x['Année de réalisation'].split():
      if s.isdigit():
         year=int(s)
    if year==None:
      return year
    elif year >2019:
      #print(year)
      year=2010
      return year
    elif year > 100:
      return int(year)
    elif year >70:
      year=int('19'+str(year))
      return int(year)
    else:
      year=None
    
  return year

In [0]:
'''
Function to change latitude/longitude data from degrees,minutes and seconds into decimals
'''
def decimals_lat_lon(x,lat):
  coord=None
  if x[lat]:
    if len(x[lat])>2:
      if len(re.split('[°\'"]', x[lat])) == 4 :
        
        #gets all the parts of the coordinates into different elements of a vector
        deg, minutes, seconds, direction =  re.split('[°\'"]', x[lat])
        
        #had to put replace commas because of data of 2009
        coord=(float(deg) + float(minutes.replace(",","."))/60 + float(seconds.replace(",","."))/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
      elif len(re.split('[°\'"]', x[lat])) > 0:
        coord=x[lat]
        print(coord)
  else:
    coord=None
    print(coord)
  return coord

#### Begin with 2019

In [0]:
#Create decimals from degrees latitude
df_2019['Latitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)

None
None


In [0]:
df_2019['Longitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

None
None


In [0]:
df_2019_filtered = df_2019[['Arrondissement', 'Village','Localite', 'Longitude_decimals',
        'Latitude_decimals', 'rqtListe_Pompe.Modele_Pompe']]
df_2019_filtered.columns = ['Arrondissement', 'Village','Localite', 'Longitude',
        'Latitude', 'Marque']

In [0]:
df_2019_filtered['Marque']=df_2019_filtered.apply(lambda x: marques(x,"Marque"),axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [0]:
l_2019_local = df_2019_filtered['Localite'].sort_values().tolist()

index = []
j=1
for i,local in enumerate(l_2019_local):
  if i == 0:
    j=1
    word = local+' '+str(j)
    word=word.replace(' ','_')
    index.append(word)
    continue
  if l_2019_local[i-1] == local:
      j+=1
      word = local+' '+str(j)
      word=word.replace(' ','_')
      index.append(word)
  else:
      j=1
      word = local+' '+str(j)
      word=word.replace(' ','_')
      index.append(word)

In [0]:
df_2019_filtered=df_2019_filtered.sort_values('Localite')

In [0]:
df_2019_filtered['index'] = index

In [0]:
df_2019_filtered=df_2019_filtered.set_index('index')

In [0]:
#convert to dictionary
dict_2019 = df_2019_filtered.to_dict('index')
dict_2019 = [{k: v} for k, v in dict_2019.items()]

##Process data of Lisette

In [10]:
df_lisette.head()

,Departement,Commune,Arrondissement,Village,Localite,Nom_Local,Longitude,Latitude,xCoord,yCoord,Type_Exhaure,Modele_Pompe
0,BORGOU,NIKKI,BIRO,BIRO,BIRO NOUKOUROU,,"02°56'50.96""E","09°54'18.50""N",2.947489,9.905139,PMH,VERGNET 4C
1,BORGOU,NIKKI,BIRO,BIRO,GOUTTO,,"02°56'47.38""E","09°54'59.99""N",2.946494,9.916664,PMH,VERGNET 4D
2,BORGOU,NIKKI,BIRO,BIRO,SARAVONDO,,"02°58'29.33""E","09°54'13.30""N",2.974814,9.903694,PMH,VERGNET 4D
3,BORGOU,NIKKI,BIRO,BIRO,MADINA,,"02°56'44.34""E","09°54'02.71""N",2.94565,9.900753,PMH,VERGNET 4C
4,BORGOU,NIKKI,BIRO,BIRO,MADINA,,"02°56'46.83""E","09°54'01.55""N",2.946342,9.900431,PMH,VERGNET 4C


In [0]:
#Create decimals from degrees latitude
df_lisette['Latitude_decimals']=df_lisette.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)

In [0]:
df_lisette['Longitude_decimals']=df_lisette.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

In [0]:
df_lisette['Marque']=df_lisette.apply(lambda x: marques(x,"Modele_Pompe","Type_Exhaure"),axis=1)

In [49]:
df_lisette.groupby("Marque",axis=0).count()

,Departement,Commune,Arrondissement,Village,Localite,Nom_Local,Longitude,Latitude,xCoord,yCoord,Type_Exhaure,Modele_Pompe,Latitude_decimals,Longitude_decimals
Marque,,,,,,,,,,,,,,
Afridev,33,33,33,33,33,33,33,33,33,33,33,33,33,33
India,95,95,95,95,95,95,95,95,95,95,95,95,95,95
NA,4,4,4,4,4,4,4,4,4,4,3,0,4,4
Vergnet,253,253,253,253,253,253,253,253,253,253,253,253,253,253
pompe_a_moteur,14,14,14,14,14,14,14,14,14,14,14,0,14,14


###Analysis to check

###How many villages have pompes

In [15]:
len(df_lisette.groupby("Localite",axis=0).count())

277

In [17]:
len(df_2009.groupby("Localité",axis=0).count())

326

#VILLAGES


In [0]:
#get data of RPGH4
RANGE_NAME = 'Sheet1'

gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID6, range=RANGE_NAME).execute()

df_rpgh4 = pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0])

##State DB

In [0]:
df_dep = df_rpgh4[df_rpgh4['Divison administrative'].str[0:4] == 'DEP:'].copy()

In [0]:
df_dep=df_dep.reset_index()

In [0]:
df_dep['Departement'] = df_dep.apply(lambda x: x['Divison administrative'][5:], axis=1)

In [0]:
df_dep=df_dep.sort_values('Departement')

In [0]:
df_dep = df_dep[['Departement','Total','Masculin','Feminin']]

### get localisation from geocoder

In [0]:
#!pip install geocoder

In [0]:
import geocoder

In [0]:
def get_coor(x):
  g=geocoder.geonames(x['Departement']+' Benin',key='5danalfaro')
  return [g.lat,g.lng]

In [0]:
df_dep['Coordinates'] = df_dep.apply(lambda x: get_coor(x),axis=1)

In [0]:
df_dep['Latitude'] = df_dep.apply(lambda x: x['Coordinates'][0],axis=1)

In [0]:
df_dep['Longitude'] = df_dep.apply(lambda x: x['Coordinates'][1],axis=1)

In [0]:
df_dep=df_dep[['Departement','Latitude','Longitude','Total']]

In [0]:
df_dep.columns = ['Name','Latitude','Longitude','Population']

Check if everything is okay, exported to csv and uploaded to google mymaps

In [0]:
df_dep.to_csv('dep.csv')

https://drive.google.com/open?id=1yJeiKBrO8j9GE10wYyRmjnQqxufG-YT0&usp=sharing all alright

In [0]:
#convert to dictionary
states = df_dep.to_dict('index')
#states = [{k: v} for k, v in dict_2019.items()]

In [0]:
states_dict = [{k: v} for k, v in states.items()] 

In [0]:
states_l_dict = []
for data in states_dict:
  id=list(data.keys())[0]
  new_data={'Name':data[id]['Name'],
            'Coordinates':{
                'Longitude':data[id]['Longitude'],
                'Latitude':data[id]['Latitude']},
            'Population':data[id]['Population']}
  states_l_dict.append(new_data)

#### upload to Firebase STATES

In [0]:
for data in states_l_dict:
  db.collection(u'LocationsDB').add(data)